In [1]:
import numpy as np
import pandas as pd
oof_lgb=pd.read_csv("train.csv")
oof_lgb=oof_lgb['label']

oof_cat=pd.read_csv("cat_train.csv")
oof_cat=oof_cat['label']
print(oof_cat)
predictions_lgb=pd.read_csv("test.csv")
predictions_lgb=predictions_lgb['label']

predictions_cat=pd.read_csv("cat_test.csv")
predictions_cat=predictions_cat['label']
print(predictions_cat)

target=pd.read_csv("new_data.csv")
target=target[~target['y1_is_purchase'].isnull()]['y1_is_purchase']
# print(target)

test=pd.read_csv("new_data.csv")
test=test[test['y1_is_purchase'].isnull()]

0         0.996404
1         0.855963
2         0.885440
3         0.549622
4         0.996682
            ...   
684278    0.290628
684279    0.683835
684280    0.450370
684281    0.598003
684282    0.839114
Name: label, Length: 684283, dtype: float64
0         0.658797
1         0.145559
2         0.674763
3         0.793671
4         0.687588
            ...   
186920    0.992612
186921    0.660384
186922    0.931174
186923    0.784321
186924    0.522966
Name: label, Length: 186925, dtype: float64


In [2]:
from sklearn.model_selection import StratifiedKFold, KFold,RepeatedKFold
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# 将lgb和cat的结果进行stacking（叠加）
train_stack = np.vstack([oof_lgb,oof_cat]).transpose()#训练集2列特征
test_stack = np.vstack([predictions_lgb, predictions_cat]).transpose()#测试集2列特征

#贝叶斯分类器也使用交叉验证的方法，5折，重复2次，主要是避免过拟合
folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2018)
oof_stack = np.zeros(train_stack.shape[0])#存放训练集中验证集的预测结果
predictions = np.zeros(test_stack.shape[0])#存放测试集的预测结果

#enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):#target就是每一行样本的标签值
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values#划分训练集的80%
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values#划分训练集的20%做验证集
    
    clf_3 = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=2021, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

    clf_3.fit(trn_data, trn_y)#贝叶斯训练过程，sklearn中的。
    
    oof_stack[val_idx] = clf_3.predict_proba(val_data)[:,1]    #对验证集有一个预测，用于后面计算模型的偏差
    predictions += clf_3.predict_proba(test_stack)[:,1]  / 10 #对测试集的预测，除以10是因为5折交叉验证重复了2次
    
print("AUC:  ",roc_auc_score(target,oof_stack))


fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
AUC:   0.9032368201722707


In [3]:
test['label']=predictions
res=test[['carid','label']]
print(res)
res.to_csv("stacking.csv",index=None)

                                               carid     label
684283  ZhCSxFMK2Pv4mkVwSNQeF0HwKQpm78mD4OA4t/gX79k=  0.699785
684284  MN3k6VEj1c1yflYHmZbnChWcB4YFunAJxn2a/oNibMo=  0.080271
684285  q0hlS2GQL/TvAvJ4QaNYOmnostsrxOd47UguA9jfsL0=  0.718266
684286  vq73YiH8neXWAG6GLRqKNmtbcMt6VexhGjdGHpqdVTg=  0.840712
684287  IAJ37++ziuqd5wAgRKP5maZjY0hmgqo5D5Wi1CENlsM=  0.676584
...                                              ...       ...
871203  3YvjqL1OhRDsNbyYEQlRUXkvdLWrmLIxBtJOWIFBmEQ=  0.949164
871204  7eHRyIkXf7QgWEBAwakeaagN52tac6USkl5X0ltVvDw=  0.683049
871205  2pZ0itTxXnUUgJMwOVbF6mrk2MYt+4esW/9UYP+rgoQ=  0.916347
871206  zeVjdYP9PCFYHPeLGgq5pBGKGrAqaSVFE3kNmjMGAAU=  0.816768
871207  rVEoH2ZFRaT/E4z9V96Y3qdUYGvelTEqcd51oHeC0Fc=  0.529543

[186925 rows x 2 columns]
